In [1]:
import os
import glob
import nltk
from nltk.corpus import stopwords

In [268]:
import numpy as np
import pandas as pd
import copy
import hashlib
import string
import itertools

In [5]:
###Preprocessing

stop_word = set(stopwords.words("english"))
punctuation_table = str.maketrans({key: " " for key in string.punctuation})
whitespace_table = str.maketrans({key: "" for key in string.whitespace})

def preprocess(document):
    
    document = document.casefold()
    document_split = (document.translate(punctuation_table)).split()
    document = ""
    for word in document_split:
        if word not in stop_word:
            document += word
    return document

In [192]:
%%time
### Form a dictionary of shingles

cwd = os.getcwd()
doc_id = {}
shingles = {}
shingles_hash = {}
empty_shingle_presence = np.zeros(100)
k = 8
for i,file in enumerate(glob.glob(cwd + "/corpus-final09/*/*.txt")):
    document = preprocess((open(file, 'rb').read()).decode('utf-8','ignore'))
    doc_id[i] = file
    for index in range(len(document) - k + 1):
        shingle = document[index: index + k]
        if shingle not in shingles.keys():
            shingles[shingle] = copy.deepcopy(empty_shingle_presence)
        shingles[shingle][i] = 1
        shingles_hash[shingle] = int((hashlib.md5(shingle.encode())).hexdigest(), 16)

CPU times: user 396 ms, sys: 12 ms, total: 408 ms
Wall time: 967 ms


In [263]:
len(shingles.keys())

37142

In [8]:
def find_perm(payload, xor):
    num1 = int((hashlib.md5(payload)).hexdigest(), 16)
    return num1 ^ xor

In [276]:
%%time

### Find 100 permutations of the shingle
### Perform minhashing

hash_functions = 240
document_count = 100
signature_matrix = np.ones((hash_functions, document_count)) * np.inf

np.random.seed(101)
perm = np.arange(len(shingles.keys()))
for i in range(hash_functions):
    np.random.shuffle(perm)
    print(i, end = "\r")
    for j,key in enumerate(shingles.keys()):
        for k in range(document_count):
            if shingles[key][k] == 1 and signature_matrix[i][k] > perm[j]:
                signature_matrix[i][k] = perm[j]
    

CPU times: user 5min 50s, sys: 71.4 ms, total: 5min 50s
Wall time: 5min 50s


In [196]:
# %%time

# hash_functions = 240
# document_count = 100
# signature_matrix_new = np.ones((hash_functions, document_count)) * np.inf

# np.random.seed(101)
# for i in range(hash_functions):
#     print(i, end = "\r")
#     xor = int(np.random.rand() * pow(10,9))
#     for shingle in shingles.keys():
#         perm = shingles_hash[shingle] ^ xor
#         for j in range(document_count):
#             if shingles[shingle][j] == 1 and signature_matrix_new[i][j] > perm:
#                 signature_matrix_new[i][j] = perm

CPU times: user 6min 53s, sys: 1.59 s, total: 6min 55s
Wall time: 6min 54s


In [277]:
signature_matrix.shape

(240, 100)

In [264]:
#signature_matrix.tofile("SignatureMatrixWith240Shingles=37142k=8")


In [ ]:
# sm = np.fromfile("SignatureMatrixWith240Shingles=37142k=8")
# sm.reshape(240,100)

In [235]:
def split_signature_matrix(signature_matrix, buckets, rows):
    divided_signature_matrix = []
    for i in range(0,signature_matrix.shape[0],rows):
        row = []
        for j in range(signature_matrix.shape[1]):
            element = signature_matrix[i:i+rows, j]
            row.append(element)
        divided_signature_matrix.append(np.asarray(row))
    return np.asarray(divided_signature_matrix)

In [236]:
def find_jaccard_similarity(set1, set2):
    intersection = 0
    for i in range(len(set1)):
        if set1[i] == set2[i]:
            intersection += 1
    return intersection/len(set1)

In [237]:
def find_euclidean_distance(set1, set2):
    dist = 0
    for i in range(set1):
        dist += (set1[i] - set2[i])**2
    return np.sqrt(dist)

In [265]:
###Generate random vectors for cosine similarity of given array size

def form_random_vectors(size):
    x = []
    for j in [1,-1]:
        for i in itertools.repeat(j,size):
            x.append(i)
    vectors_set = set({})
    for i in itertools.permutations(x, size):
        vectors_set.add(i)
    random_vectors = []
    for v in vectors_set:
        vect = []
        for i in v:
            vect.append(i)
        random_vectors.append(vect)
    return np.asarray(random_vectors)

In [321]:
def cosine_distance(set1, set2, random_vectors):
#     similarity = 0
#     for vect in random_vectors:
#         dot1 = (np.dot(set1, vect) > 0)
#         dot2 = (np.dot(set2, vect) > 0)
#         if dot1 == dot2:
#             similarity += 1
    len1 = 0
    len2 = 0
    for i in range(len(set1)):
        len1 += set1[i]**2
        len2 += set2[i]**2
    len_prod = (len1 * len2)**0.5
    return np.arccos((np.dot(set1, set2)) / len_prod) * 180/np.pi
#     return similarity/len(random_vectors)

In [320]:
np.dot(signature_matrix_split[0,1], signature_matrix_split[0,2])

2844.0

In [322]:
%%time

### Find Similarity 
### Define number of buckets and rows

buckets = 60
rows = 4
print("rows = "  + str(rows))
jaccard_similarity_matrix = np.zeros((document_count, document_count))
signature_matrix_split = split_signature_matrix(signature_matrix, buckets, rows)

### Jaccard Similarity
for i in range(document_count):
    print(i, end = "\r")
    for j in range(document_count):
        for k in range(buckets):
            similar = find_jaccard_similarity(signature_matrix_split[k,i], signature_matrix_split[k,j])
            jaccard_similarity_matrix[i][j] = max(jaccard_similarity_matrix[i][j], similar)
            jaccard_similarity_matrix[j][i] = jaccard_similarity_matrix[i][j]

### Cosine distance
random_vectors = form_random_vectors(rows)
cosine_similarity_matrix = np.ones((document_count, document_count)) * np.inf
for i in range(document_count):
    print(i, end = "\r")
    for j in range(document_count):
        for k in range(buckets):
            distance = cosine_distance(signature_matrix_split[k,i], signature_matrix_split[k,j], random_vectors)
            cosine_similarity_matrix[i][j] = min(cosine_similarity_matrix[i][j], distance)
            cosine_similarity_matrix[j][i] = cosine_similarity_matrix[i][j]


rows = 4
CPU times: user 10.6 s, sys: 847 ms, total: 11.5 s
Wall time: 9.95 s


In [311]:
setup_df = pd.read_excel("corpus-final09(copy).xls")
doc_plagiarism_type = {}
for i,file in enumerate(setup_df["File"]):
    doc_plagiarism_type[file] = setup_df.iloc[i,4]

In [312]:
original_doc_id = []
for doc in doc_id.keys():
    if doc_id[doc][-14: -10] == "orig":
        original_doc_id.append(doc)

In [313]:
tp = "True_Positives"
fp = "False_Positives"
tn = "True_Negative"
fn = "False_Negative"

In [314]:
cut = "cut"
heavy = "heavy"
light = "light"
non = "non"

In [317]:
cosine_similarity_matrix

array([[inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf]])

In [289]:
###Jacard Similarity
similar_documents = {}
results = {}
performance = {}
results[tp] = results[fp] = results[fn] = results[tn] = results[cut] = results[heavy] = results[light] = results[non] = 0
threshold = 0.8

for k in original_doc_id:    
    similar_documents[ doc_id[k][-5] ] = []
    performance[ doc_id[k][-5] ] = copy.deepcopy(results) 
    
    for i in range(len( jaccard_similarity_matrix[k] )):
        if jaccard_similarity_matrix[k][i] >= threshold and k != i:
            similar_documents[ doc_id[k][-5] ].append(i)
            if doc_id[i][-5] == doc_id[k][-5]:
                performance[doc_id[k][-5]][tp] += 1
                performance[doc_id[k][-5]][doc_plagiarism_type[doc_id[i][-14:]]] += 1
            else:
                performace[doc_id[k][-5]][fp] += 1
    performance[doc_id[k][-5]][tn] = 19 - performance[doc_id[k][-5]][tp]
    performance[doc_id[k][-5]][fn] = 80 - performance[doc_id[k][-5]][fp]

In [338]:
###Cosine distance
similar_documents = {}
results = {}
performance = {}
results[tp] = results[fp] = results[fn] = results[tn] = results[cut] = results[heavy] = results[light] = results[non] = 0
threshold = 4

for k in original_doc_id:    
    similar_documents[ doc_id[k][-5] ] = []
    performance[ doc_id[k][-5] ] = copy.deepcopy(results) 
    
    for i in range(len( jaccard_similarity_matrix[k] )):
        if cosine_similarity_matrix[k][i] <= threshold and k != i:
            similar_documents[ doc_id[k][-5] ].append(i)
            if doc_id[i][-5] == doc_id[k][-5]:
                performance[doc_id[k][-5]][tp] += 1
                performance[doc_id[k][-5]][doc_plagiarism_type[doc_id[i][-14:]]] += 1
            else:
                performace[doc_id[k][-5]][fp] += 1
    performance[doc_id[k][-5]][tn] = 19 - performance[doc_id[k][-5]][tp]
    performance[doc_id[k][-5]][fn] = 80 - performance[doc_id[k][-5]][fp]

for doc in similar_documents.keys():
    doc

KeyError: 'c'

In [336]:
similar_documents

{'c': [3, 17, 26, 39, 40, 41, 48, 50, 67, 71, 93, 94, 96],
 'd': [0, 15, 19, 30, 34, 37, 42, 51, 53, 65, 79, 86, 87],
 'a': [0, 18, 25, 30, 37, 41, 44, 46, 49, 54, 57, 59, 62, 72, 73, 85, 89, 98],
 'b': [5, 20, 22, 46, 49, 56, 57, 66, 80, 91, 94],
 'e': [8, 11, 21, 24, 35, 36, 59, 72, 85, 86]}

In [333]:
df = pd.DataFrame(performance).T

In [334]:
df

,True_Positives,False_Positives,False_Negative,True_Negative,cut,heavy,light,non
c,10,0,80,9,2,4,3,1
d,8,0,80,11,3,2,3,0
a,9,0,80,10,4,1,3,1
b,5,0,80,14,2,1,1,1
e,7,0,80,12,5,0,2,0


In [335]:
cosine_similarity_matrix[1]

array([ 5.78007077,  0.        ,  7.78357544,  0.        , 12.26619318,
        7.99076388,  8.31051739,  8.41310731, 10.85570433,  8.6715246 ,
       10.87615345,  5.94337554,  7.07357814, 16.9327643 , 15.1517795 ,
       14.66839722,  6.80740357,  3.74891757, 11.47131916, 12.76057536,
        4.75088613,  6.38789568, 12.34309758, 12.21754672,  7.94373057,
        7.95287434,  0.        , 10.45496015,  8.3701498 , 10.52064742,
        8.59259576,  4.32849338, 14.29548991, 10.01701805,  5.61697779,
        8.87840951,  5.48672841,  6.2131979 ,  9.40065357,  0.        ,
        2.04571724,  2.06343262,  8.80961937,  8.20377549,  5.01759599,
        8.93810672,  8.21537835, 10.15806766,  2.15972049, 18.57356754,
        0.        ,  8.27738742,  9.78819496,  5.87098135,  4.60229155,
       17.4733508 , 20.102668  , 16.21157461, 17.95436661, 11.7516797 ,
        5.5733518 ,  5.26411227, 12.37382953,  9.09571682, 16.11383669,
        5.56250116,  4.55622734,  0.95122228,  4.75531729,  5.06

array([ 0.        ,  5.78007077,  5.415753  ,  7.42715756,  7.78260694,
       13.85413917,  5.59749778,  8.64181959,  8.36384193,  0.34110989,
        6.4334432 , 11.27109575, 15.71278966,  3.35081684, 12.77361282,
        5.04185638, 10.31195471, 12.01447978,  1.81610069,  5.67688143,
        7.58472376,  5.92197518,  8.48697859,  9.16004406,  4.52445888,
        5.5234168 ,  5.78007077,  6.44423782,  8.59155084, 11.86125246,
        8.4984848 , 10.92695742,  3.53075199,  8.07522298,  5.85151703,
        4.52445888, 10.81078666,  9.0014286 ,  8.78297965, 11.20909221,
        5.04252914, 11.2036198 ,  0.34110989,  4.24451325, 15.10432312,
        6.43362216,  8.15324621,  3.88957873, 12.0657095 ,  3.6912321 ,
       10.65808807, 10.18483404, 14.37911851,  0.34110989, 15.64470234,
       12.41460979,  3.6912321 ,  3.6912321 ,  7.94835071,  7.08110639,
        3.52025245,  8.08874503, 16.51446835,  6.46703149,  2.9892823 ,
       11.79603519,  4.7291318 , 10.13315843,  1.60547345, 14.17